In [1]:
import requests
import pandas as pd
import numpy as np

# Get the data

In [2]:
# fuction to get the data 


# Function to download a file from a URL and save it in the `data` folder
def download_file(url, local_filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(local_filename, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded {local_filename}")
    else:
        print(f"Failed to download {local_filename}. Status code: {response.status_code}")

url_orders = 'https://raw.githubusercontent.com/Digital-IFCO/data-engineering-test/main/resources/orders.csv'
url_invoicing = 'https://raw.githubusercontent.com/Digital-IFCO/data-engineering-test/main/resources/invoicing_data.json'

# Download the files into the `data` folder
download_file(url_orders, '../data/orders.csv')
download_file(url_invoicing, '../data/invoicing_data.json')

Downloaded ../data/orders.csv
Downloaded ../data/invoicing_data.json


In [3]:

# Cargar el archivo orders.csv
orders = pd.read_csv('../data/orders.csv',delimiter=';')

# Verificar las primeras filas para revisar la estructura de los datos
orders.head()

,order_id,date,company_id,company_name,crate_type,contact_data,salesowners
0,f47ac10b-58cc-4372-a567-0e02b2c3d479,29.01.22,1e2b47e6-499e-41c6-91d3-09d12dddfbbd,Fresh Fruits Co,Plastic,"[{ ""contact_name"":""Curtis"", ""contact_surname"":...","Leonard Cohen, Luke Skywalker, Ammy Winehouse"
1,f47ac10b-58cc-4372-a567-0e02b2c3d480,21.02.22,0f05a8f1-2bdf-4be7-8c82-4c9b58f04898,Veggies Inc,Wood,"[{ ""contact_name"":""Maria"", ""contact_surname"":""...","Luke Skywalker, David Goliat, Leon Leonov"
2,f47ac10b-58cc-4372-a567-0e02b2c3d481,03.04.22,1e2b47e6-499e-41c6-91d3-09d12dddfbbd,Fresh Fruits c.o,Metal,"[{ ""contact_name"":""Para"", ""contact_surname"":""C...",Luke Skywalker
3,f47ac10b-58cc-4372-a567-0e02b2c3d482,14.07.21,1c4b0b50-1d5d-463a-b56e-1a6fd3aeb7d6,Seafood Supplier,Plastic,NaN,"David Goliat, Leonard Cohen"
4,f47ac10b-58cc-4372-a567-0e02b2c3d483,23.10.22,34538e39-cd2e-4641-8d24-3c94146e6f16,Meat Packers Ltd,Plastic,NaN,"Chris Pratt, David Henderson, Marianov Merschi..."


# Distribution of Crate Type per Company


In [4]:
import pandas as pd
import logging

# Configurar el logging para mostrar advertencias
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Cargar el archivo orders.csv con el delimitador adecuado
orders = pd.read_csv('../data/orders.csv', delimiter=';')

# Verificar y manejar valores faltantes en 'company_id' y 'crate_type'
if orders['company_id'].isnull().any():
    logging.warning("Some rows have missing 'company_id'. These rows will be excluded from the distribution calculation.")
    orders = orders.dropna(subset=['company_id'])  # Excluir filas sin 'company_id'

if orders['crate_type'].isnull().any():
    logging.warning("Some rows have missing 'crate_type'. These rows will be excluded from the distribution calculation.")
    orders = orders.dropna(subset=['crate_type'])  # Excluir filas sin 'crate_type'

# Verificar tipos de datos en 'company_id' y 'crate_type'
if not pd.api.types.is_string_dtype(orders['company_id']):
    logging.warning("'company_id' is not of string type. Converting to string.")
    orders['company_id'] = orders['company_id'].astype(str)

if not pd.api.types.is_string_dtype(orders['crate_type']):
    logging.warning("'crate_type' is not of string type. Converting to string.")
    orders['crate_type'] = orders['crate_type'].astype(str)

# Lista de valores permitidos en 'crate_type'
valid_crate_types = ["Plastic", "Wood", "Metal"]

# Verificar si 'crate_type' contiene valores fuera de los permitidos
invalid_crate_types = orders[~orders['crate_type'].isin(valid_crate_types)]
if not invalid_crate_types.empty:
    logging.warning(f"Some rows contain invalid crate types. These will be excluded from the calculation.\n{invalid_crate_types['crate_type'].unique()}")
    orders = orders[orders['crate_type'].isin(valid_crate_types)]

# Comprobación de duplicados en 'order_id'
if orders['order_id'].duplicated().any():
    logging.warning("Some rows contain duplicate 'order_id'. These rows may indicate duplicate orders.")

# Calcular la distribución de tipos de cajas por empresa después de las verificaciones
distribution = orders.groupby(['company_id', 'crate_type']).size().reset_index(name='order_count')

# Mostrar los primeros resultados para verificar el cálculo
distribution.head()

,company_id,crate_type,order_count
0,012f20c6-00d5-4f45-999f-12e7639db623,Plastic,1
1,063a7dc7-b93a-4f38-b7f0-0e30b5b217ac,Wood,1
2,0b8755d4-3d28-4039-b9a7-b30cb5ff02ea,Wood,1
3,0d09ae2b-d9a5-4d67-bb97-963be9379b4e,Plastic,1
4,0f05a8f1-2bdf-4be7-8c82-4c9b58f04898,Plastic,1


## task 2




In [5]:
import json

def get_full_name(contact_data):
    """Extracts full name from contact data or returns 'John Doe' if incomplete."""
    try:
        # Load the contact data as a JSON object
        contact_info = json.loads(contact_data)[0]  # Access the dictionary inside the list
        first_name = contact_info.get("contact_name", "").strip()
        last_name = contact_info.get("contact_surname", "").strip()
        
        # If either first or last name is missing, default to "John Doe"
        if not first_name or not last_name:
            return "John Doe"
        return f"{first_name} {last_name}"
    except (json.JSONDecodeError, IndexError, TypeError, KeyError):
        # In case of unexpected structure, default to "John Doe"
        return "John Doe"

In [6]:
import pandas as pd

# Load data
file_path = '../data/orders.csv'
orders_df = pd.read_csv(file_path, delimiter=';')

# Function to extract the full name
import json

def get_full_name(contact_data):
    """Extracts full name from contact data or returns 'John Doe' if incomplete."""
    try:
        # Load the contact data as a JSON object
        contact_info = json.loads(contact_data)[0]  # Access the dictionary inside the list
        first_name = contact_info.get("contact_name", "").strip()
        last_name = contact_info.get("contact_surname", "").strip()
        
        # If either first or last name is missing, default to "John Doe"
        if not first_name or not last_name:
            return "John Doe"
        return f"{first_name} {last_name}"
    except (json.JSONDecodeError, IndexError, TypeError, KeyError):
        # In case of unexpected structure, default to "John Doe"
        return "John Doe"

# Apply the function to create contact_full_name
orders_df['contact_full_name'] = orders_df['contact_data'].apply(get_full_name)

# Select relevant columns
contact_fullname_df = orders_df[['order_id', 'contact_full_name']]

# Display the resulting DataFrame
contact_fullname_df.head()

,order_id,contact_full_name
0,f47ac10b-58cc-4372-a567-0e02b2c3d479,Curtis Jackson
1,f47ac10b-58cc-4372-a567-0e02b2c3d480,Maria Theresa
2,f47ac10b-58cc-4372-a567-0e02b2c3d481,Para Cetamol
3,f47ac10b-58cc-4372-a567-0e02b2c3d482,John Doe
4,f47ac10b-58cc-4372-a567-0e02b2c3d483,John Doe


In [7]:
import pandas as pd
import json
import os
import logging
import re

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def load_orders(file_path, delimiter=';'):
    """Loads order data from a CSV file."""
    try:
        df = pd.read_csv(file_path, delimiter=delimiter)
        logging.info(f"Loaded {len(df)} rows from {file_path}")
        return df
    except FileNotFoundError:
        logging.error(f"Error: The file {file_path} was not found.")
        return pd.DataFrame()

def fix_contact_data_format(contact_data):
    """Attempts to fix known format issues in contact_data strings."""
    if pd.isna(contact_data):
        return None
    # Ensure JSON-compatible format
    contact_data = re.sub(r'([{,])\s*([a-zA-Z_]+)\s*:', r'\1 "\2":', contact_data)
    contact_data = contact_data.replace("'", '"')
    if not contact_data.startswith('['):
        contact_data = f"[{contact_data}]"
    contact_data = re.sub(r'}\s*{', '},{', contact_data)
    logging.info(f"Formatted contact_data: {contact_data}")
    return contact_data

def get_full_name(contact_data):
    """Extracts full name from contact data or returns 'John Doe' if incomplete."""
    contact_data = fix_contact_data_format(contact_data)
    if contact_data is None:
        return "John Doe"
    try:
        contact_info = json.loads(contact_data)[0]
        first_name = contact_info.get("contact_name", "").strip()
        last_name = contact_info.get("contact_surname", "").strip()
        if not first_name or not last_name:
            return "John Doe"
        return f"{first_name} {last_name}"
    except (json.JSONDecodeError, IndexError, TypeError, KeyError) as e:
        logging.warning(f"Unexpected structure in contact data: {contact_data}. Error: {e}. Using 'John Doe'.")
        return "John Doe"

def create_contact_fullname_df(df):
    """Creates a DataFrame with order_id and contact_full_name."""
    if df.empty:
        logging.warning("The DataFrame is empty. No contact full names to process.")
        return pd.DataFrame(columns=['order_id', 'contact_full_name'])
    df['contact_full_name'] = df['contact_data'].apply(get_full_name)
    logging.info("Successfully created contact_full_name column.")
    return df[['order_id', 'contact_full_name']]

def save_contact_fullname_df(df, output_path):
    """Saves the contact full name DataFrame to a CSV file."""
    if df.empty:
        logging.warning("The DataFrame is empty. Nothing to save.")
        return
    absolute_path = os.path.abspath(output_path)
    logging.info(f"Saving DataFrame to {absolute_path}")
    os.makedirs(os.path.dirname(absolute_path), exist_ok=True)
    df.to_csv(absolute_path, index=False)
    logging.info(f"DataFrame successfully saved to {absolute_path}")
    if os.path.exists(absolute_path):
        logging.info(f"File {absolute_path} was created successfully.")
    else:
        logging.error(f"File {absolute_path} was not created.")

if __name__ == "__main__":
    # Print current working directory for diagnostics
    logging.info(f"Current working directory: {os.getcwd()}")
    # Load and process data
    orders_df = load_orders('../data/orders.csv')
    if not orders_df.empty:
        contact_fullname_df = create_contact_fullname_df(orders_df)
        save_contact_fullname_df(contact_fullname_df, '../output/contact_fullname.csv')
    else:
        logging.error("No data loaded. Exiting.")

2024-11-05 21:06:41,987 - INFO - Current working directory: /Users/pedro/Library/CloudStorage/Dropbox/PROYECTOS/others/data-engineering-test/notebooks
2024-11-05 21:06:41,989 - INFO - Loaded 62 rows from ../data/orders.csv
2024-11-05 21:06:41,989 - INFO - Formatted contact_data: [{ "contact_name":"Curtis", "contact_surname":"Jackson", "city":"Chicago", "cp": "12345"}]
2024-11-05 21:06:41,990 - INFO - Formatted contact_data: [{ "contact_name":"Maria", "contact_surname":"Theresa", "city":"Calcutta"}]
2024-11-05 21:06:41,990 - INFO - Formatted contact_data: [{ "contact_name":"Para", "contact_surname":"Cetamol", "city":"Frankfurt am Oder", "cp": 3934}]
2024-11-05 21:06:41,991 - INFO - Formatted contact_data: [{ "contact_name":"John", "contact_surname":"Krasinski", "city":"New York", "cp": "1203"}]
2024-11-05 21:06:42,032 - INFO - Formatted contact_data: [{ "contact_name":"Jennifer", "contact_surname":"Lopez", "city":"Esplugues de Llobregat"}]
2024-11-05 21:06:42,041 - INFO - Formatted cont

$~$

modifying the function to get more robust solution

$~$

In [8]:
import pandas as pd
import json
import os
import logging
import re

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def load_orders(file_path, delimiter=';'):
    """Loads order data from a CSV file."""
    try:
        df = pd.read_csv(file_path, delimiter=delimiter)
        logging.info(f"Loaded {len(df)} rows from {file_path}")
        return df
    except FileNotFoundError:
        logging.error(f"Error: The file {file_path} was not found.")
        return pd.DataFrame()  # Return an empty DataFrame if the file is missing

def fix_contact_data_format(contact_data):
    """Intenta corregir y validar el formato de contact_data de manera más detallada."""
    if pd.isna(contact_data):
        return None

    # Reemplazar comillas simples por dobles y asegurarse de que todas las claves estén en comillas dobles
    contact_data = re.sub(r"([{\[,]\s*)(\w+)(\s*:\s*)", r'\1"\2"\3', contact_data.replace("'", '"'))

    # Añadir comillas a los valores si faltan y no son numéricos o booleanos
    contact_data = re.sub(r":\s*([a-zA-Z_]+)([,\]}\s])", r': "\1"\2', contact_data)

    # Asegurarse de que el JSON esté entre corchetes
    if not contact_data.startswith('['):
        contact_data = f'[{contact_data}]'
    
    # Corregir problemas comunes de formato como objetos JSON mal concatenados
    contact_data = re.sub(r'}\s*,\s*{', '},{', contact_data)
    
    try:
        json.loads(contact_data)  # Verificar si el formato JSON es correcto ahora
        return contact_data
    except json.JSONDecodeError as e:
        logging.warning(f"Error en formato de contact_data ({contact_data}): {e}. Usando 'Unknown, UNK00'")
        return None

def get_full_name(contact_data):
    """Extracts full name from contact data or returns 'John Doe' if incomplete."""
    contact_data = fix_contact_data_format(contact_data)
    if contact_data is None:
        return "John Doe"
    
    try:
        contact_info = json.loads(contact_data)[0]  # Procesa como lista JSON
        first_name = contact_info.get("contact_name", "").strip()
        last_name = contact_info.get("contact_surname", "").strip()
        
        # Si falta nombre o apellido, devuelve "John Doe"
        if not first_name or not last_name:
            return "John Doe"
        return f"{first_name} {last_name}"
    except (json.JSONDecodeError, IndexError, TypeError, KeyError) as e:
        logging.warning(f"Unexpected structure in contact data: {contact_data}. Error: {e}. Using 'John Doe'.")
        return "John Doe"

def create_contact_fullname_df(df):
    """Creates a DataFrame with order_id and contact_full_name."""
    if df.empty:
        logging.warning("The DataFrame is empty. No contact full names to process.")
        return pd.DataFrame(columns=['order_id', 'contact_full_name'])

    # Aplica la función para crear la columna contact_full_name
    df['contact_full_name'] = df['contact_data'].apply(get_full_name)
    logging.info("Successfully created contact_full_name column.")
    return df[['order_id', 'contact_full_name']]

def save_contact_fullname_df(df, output_path):
    """Saves the contact full name DataFrame to a CSV file."""
    if df.empty:
        logging.warning("The DataFrame is empty. Nothing to save.")
        return

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df.to_csv(output_path, index=False)
    logging.info(f"DataFrame successfully saved to {output_path}")
    
    # Confirmación final para verificar si el archivo existe después de guardarlo
    if os.path.exists(output_path):
        logging.info(f"File {output_path} was created successfully.")
    else:
        logging.error(f"File {output_path} was not created.")

# Ejemplo de uso
if __name__ == "__main__":
    # Cargar pedidos
    orders_df = load_orders('../data/orders.csv')
    
    # Verifica si el DataFrame se cargó correctamente antes de continuar
    if not orders_df.empty:
        # Procesa nombres completos de contacto
        contact_fullname_df = create_contact_fullname_df(orders_df)
        
        # Guarda el DataFrame resultante
        save_contact_fullname_df(contact_fullname_df, '../output/contact_fullname.csv')
    else:
        logging.error("No data loaded. Exiting.")

2024-11-05 21:06:42,095 - INFO - Loaded 62 rows from ../data/orders.csv
2024-11-05 21:06:42,096 - WARNING - Error en formato de contact_data ([{ "contact_name":"Liav", "contact_surname": "Ichenbaum", "city":"Tel Aviv"}): Expecting ',' delimiter: line 1 column 77 (char 76). Usando 'Unknown, UNK00'
2024-11-05 21:06:42,097 - INFO - Successfully created contact_full_name column.
2024-11-05 21:06:42,100 - INFO - DataFrame successfully saved to ../output/contact_fullname.csv
2024-11-05 21:06:42,100 - INFO - File ../output/contact_fullname.csv was created successfully.


## Test 3

In [9]:
import pandas as pd
import json
import os
import logging
import re

# Configuración del logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def load_orders(file_path, delimiter=';'):
    """Carga los datos de órdenes desde un archivo CSV."""
    try:
        df = pd.read_csv(file_path, delimiter=delimiter)
        logging.info(f"Cargadas {len(df)} filas desde {file_path}")
        return df
    except FileNotFoundError:
        logging.error(f"Error: El archivo {file_path} no fue encontrado.")
        return pd.DataFrame()

def fix_contact_data_format(contact_data):
    """Intenta corregir problemas de formato conocidos en las cadenas de contact_data."""
    if pd.isna(contact_data) or not isinstance(contact_data, str):
        logging.debug("El valor de contact_data es NaN o no es una cadena, usando valor predeterminado.")
        return '[{"city": "Unknown", "cp": "UNK00"}]'
    
    logging.debug(f"Formato original de contact_data: {contact_data}")
    
    # Normalización inicial: agregar comillas y llaves donde falten
    contact_data = re.sub(r'([{,])\s*([a-zA-Z_]+)\s*:', r'\1 "\2":', contact_data)
    contact_data = contact_data.replace("'", '"').strip()
    logging.debug(f"Formato después de la normalización inicial: {contact_data}")
    
    # Eliminar caracteres extras al final que pueden estar generando errores
    if contact_data.endswith(']}'):
        contact_data = contact_data[:-1]
    elif contact_data.endswith('}]}'):
        contact_data = contact_data[:-2] + "}]"
    logging.debug(f"Formato después de limpiar caracteres extra: {contact_data}")

    # Asegurarse que empieza con "[" y termina con "]" para que sea una lista JSON
    if not contact_data.startswith('['):
        contact_data = f"[{contact_data}"
    if not contact_data.endswith(']'):
        contact_data += ']'
    
    # Validación de la estructura de JSON
    try:
        json.loads(contact_data)
    except json.JSONDecodeError as e:
        logging.warning(f"Error en el formato de contact_data tras la normalización: {contact_data}. Error: {e}")
        contact_data = '[{"city": "Unknown", "cp": "UNK00"}]'
    
    logging.debug(f"Formato final de contact_data: {contact_data}")
    return contact_data

def get_contact_address(contact_data):
    """Extrae la dirección de la ciudad y el código postal o devuelve valores predeterminados si faltan."""
    contact_data = fix_contact_data_format(contact_data)
    if contact_data is None:
        logging.debug("contact_data es None, devolviendo 'Unknown, UNK00'")
        return "Unknown, UNK00"
    
    try:
        contact_info = json.loads(contact_data)[0]
        city = contact_info.get("city", "Unknown").strip() if isinstance(contact_info.get("city"), str) else "Unknown"
        postal_code = contact_info.get("cp", "UNK00").strip() if isinstance(contact_info.get("cp"), str) else "UNK00"
        logging.debug(f"Extrayendo contact_address: {city}, {postal_code}")
        return f"{city}, {postal_code}"
    except (json.JSONDecodeError, IndexError, TypeError, KeyError) as e:
        logging.warning(f"Error persistente en formato de contact_data ({contact_data}): {e}. Usando 'Unknown, UNK00'")
        return "Unknown, UNK00"

def create_contact_address_df(df):
    """Crea un DataFrame con order_id y contact_address."""
    if df.empty:
        logging.warning("El DataFrame está vacío. No hay nombres de contacto para procesar.")
        return pd.DataFrame(columns=['order_id', 'contact_address'])
    
    logging.debug("Iniciando creación de columna contact_address a partir de contact_data.")
    df['contact_address'] = df['contact_data'].apply(get_contact_address)
    logging.info("Columna contact_address creada exitosamente.")
    return df[['order_id', 'contact_address']]

def save_contact_address_df(df, output_path):
    """Guarda el DataFrame de dirección de contacto en un archivo CSV."""
    if df.empty:
        logging.warning("El DataFrame está vacío. No hay nada que guardar.")
        return
    absolute_path = os.path.abspath(output_path)
    logging.info(f"Guardando DataFrame en {absolute_path}")
    output_dir = os.path.dirname(absolute_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    df.to_csv(absolute_path, index=False)
    logging.info(f"DataFrame guardado exitosamente en {absolute_path}")
    if os.path.exists(absolute_path):
        logging.info(f"Archivo {absolute_path} creado exitosamente.")
    else:
        logging.error(f"El archivo {absolute_path} no fue creado.")

# Ejecución del proceso
input_path = '../data/orders.csv'
output_path = '../output/contact_address.csv'

orders_df = load_orders(input_path)
if not orders_df.empty:
    contact_address_df = create_contact_address_df(orders_df)
    save_contact_address_df(contact_address_df, output_path)
else:
    logging.error("No se cargaron datos. Saliendo.")

2024-11-05 21:06:42,113 - INFO - Cargadas 62 filas desde ../data/orders.csv
2024-11-05 21:06:42,114 - INFO - Columna contact_address creada exitosamente.
2024-11-05 21:06:42,115 - INFO - Guardando DataFrame en /Users/pedro/Library/CloudStorage/Dropbox/PROYECTOS/others/data-engineering-test/output/contact_address.csv
2024-11-05 21:06:42,117 - INFO - DataFrame guardado exitosamente en /Users/pedro/Library/CloudStorage/Dropbox/PROYECTOS/others/data-engineering-test/output/contact_address.csv
2024-11-05 21:06:42,117 - INFO - Archivo /Users/pedro/Library/CloudStorage/Dropbox/PROYECTOS/others/data-engineering-test/output/contact_address.csv creado exitosamente.


## TASK 4

In [10]:
import pandas as pd
import json

# Cargar el archivo JSON de facturación
with open('../data/invoicing_data.json', 'r') as f:
    invoicing_data = json.load(f)["data"]["invoices"]

# Convertir datos de facturación a DataFrame y calcular el valor neto en euros
invoicing_df = pd.DataFrame(invoicing_data)
invoicing_df['grossValue'] = pd.to_numeric(invoicing_df['grossValue'])
invoicing_df['vat'] = pd.to_numeric(invoicing_df['vat'])
invoicing_df['net_invoiced_value_euros'] = invoicing_df['grossValue'] * (1 - invoicing_df['vat'] / 100) / 100

# Cargar el archivo CSV de órdenes
orders_df = pd.read_csv('../data/orders.csv', delimiter=';')

# Convertir `salesowners` a listas de nombres
orders_df['salesowners'] = orders_df['salesowners'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

# Unir los DataFrames por `order_id` y `orderId`
merged_df = orders_df.merge(
    invoicing_df[['orderId', 'net_invoiced_value_euros']], 
    left_on='order_id', 
    right_on='orderId', 
    how='left'
)

# Definir las tasas de comisión para cada posición de propietario
commission_rates = {
    "main_owner": 0.06,
    "co_owner_1": 0.025,
    "co_owner_2": 0.0095
}

# Inicializar un diccionario para acumular comisiones por cada propietario
commission_dict = {}

# Calcular las comisiones para cada orden
for _, row in merged_df.iterrows():
    salesowners = row['salesowners']
    net_value = row['net_invoiced_value_euros']
    
    # Comprobar que el valor neto no sea NaN
    if pd.notna(net_value):
        # Comisiones para el propietario principal
        if len(salesowners) > 0:
            main_owner = salesowners[0]
            main_commission = net_value * commission_rates["main_owner"]
            commission_dict[main_owner] = commission_dict.get(main_owner, 0) + main_commission

        # Comisiones para el co-propietario 1
        if len(salesowners) > 1:
            co_owner_1 = salesowners[1]
            co_owner_1_commission = net_value * commission_rates["co_owner_1"]
            commission_dict[co_owner_1] = commission_dict.get(co_owner_1, 0) + co_owner_1_commission

        # Comisiones para el co-propietario 2
        if len(salesowners) > 2:
            co_owner_2 = salesowners[2]
            co_owner_2_commission = net_value * commission_rates["co_owner_2"]
            commission_dict[co_owner_2] = commission_dict.get(co_owner_2, 0) + co_owner_2_commission

# Convertir el diccionario de comisiones a DataFrame y ordenar
commission_df = pd.DataFrame(list(commission_dict.items()), columns=['sales_owner', 'total_commission'])
commission_df = commission_df.sort_values(by='total_commission', ascending=False).reset_index(drop=True)

# Redondear la columna de comisión total a dos decimales
commission_df['total_commission'] = commission_df['total_commission'].round(2)

# Mostrar el DataFrame resultante
print("\nSales Owner Commissions:")
display(commission_df)


Sales Owner Commissions:


,sales_owner,total_commission
0,Leonard Cohen,629.28
1,David Henderson,465.34
2,Luke Skywalker,366.54
3,Yuri Gagarin,273.49
4,David Goliat,250.32
5,Ammy Winehouse,201.72
6,Marianov Merschik,188.61
7,Chris Pratt,112.90
8,Vladimir Chukov,72.83
9,Marie Curie,67.41


In [11]:
import pandas as pd
import json
import logging
import os

# Configuración del logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Definir las tasas de comisión para cada posición de propietario
commission_rates = {
    "main_owner": 0.06,
    "co_owner_1": 0.025,
    "co_owner_2": 0.0095
}

def load_invoicing_data(filepath):
    """Carga los datos de facturación desde un archivo JSON."""
    if not os.path.exists(filepath):
        logging.error(f"File {filepath} not found.")
        return pd.DataFrame()
    
    try:
        with open(filepath, 'r') as f:
            invoicing_data = json.load(f).get("data", {}).get("invoices", [])
        invoicing_df = pd.DataFrame(invoicing_data)
        logging.info(f"Loaded invoicing data with {len(invoicing_df)} rows.")
        return invoicing_df
    except json.JSONDecodeError as e:
        logging.error(f"Error loading JSON data: {e}")
        return pd.DataFrame()

def preprocess_invoicing_data(invoicing_df):
    """Convierte valores en centavos a euros y calcula el valor neto."""
    try:
        invoicing_df['grossValue'] = pd.to_numeric(invoicing_df['grossValue'])
        invoicing_df['vat'] = pd.to_numeric(invoicing_df['vat'])
        invoicing_df['net_invoiced_value_euros'] = invoicing_df['grossValue'] * (1 - invoicing_df['vat'] / 100) / 100
        logging.info("Processed invoicing data with net values in euros.")
        return invoicing_df[['orderId', 'net_invoiced_value_euros']]
    except KeyError as e:
        logging.error(f"Missing expected column in invoicing data: {e}")
        return pd.DataFrame()

def load_orders_data(filepath):
    """Carga los datos de órdenes desde un archivo CSV y convierte `salesowners` a listas."""
    if not os.path.exists(filepath):
        logging.error(f"File {filepath} not found.")
        return pd.DataFrame()
    
    try:
        orders_df = pd.read_csv(filepath, delimiter=';')
        orders_df['salesowners'] = orders_df['salesowners'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])
        logging.info(f"Loaded orders data with {len(orders_df)} rows.")
        return orders_df
    except pd.errors.ParserError as e:
        logging.error(f"Error loading CSV data: {e}")
        return pd.DataFrame()

def calculate_commissions(merged_df):
    """Calcula las comisiones para cada propietario según las tasas definidas."""
    commission_dict = {}
    
    for _, row in merged_df.iterrows():
        salesowners = row['salesowners']
        net_value = row['net_invoiced_value_euros']
        
        if pd.notna(net_value):
            if len(salesowners) > 0:
                main_owner = salesowners[0]
                commission_dict[main_owner] = commission_dict.get(main_owner, 0) + net_value * commission_rates["main_owner"]
            
            if len(salesowners) > 1:
                co_owner_1 = salesowners[1]
                commission_dict[co_owner_1] = commission_dict.get(co_owner_1, 0) + net_value * commission_rates["co_owner_1"]
                
            if len(salesowners) > 2:
                co_owner_2 = salesowners[2]
                commission_dict[co_owner_2] = commission_dict.get(co_owner_2, 0) + net_value * commission_rates["co_owner_2"]
    
    return pd.DataFrame(list(commission_dict.items()), columns=['sales_owner', 'total_commission'])

def main():
    # Rutas de archivos
    invoicing_filepath = '../data/invoicing_data.json'
    orders_filepath = '../data/orders.csv'
    
    # Cargar y procesar datos de facturación
    invoicing_df = load_invoicing_data(invoicing_filepath)
    invoicing_df = preprocess_invoicing_data(invoicing_df)
    
    # Cargar datos de órdenes
    orders_df = load_orders_data(orders_filepath)
    
    if invoicing_df.empty or orders_df.empty:
        logging.error("One or more required datasets are empty. Exiting.")
        return
    
    # Unir los DataFrames por `order_id` y `orderId`
    merged_df = orders_df.merge(invoicing_df, left_on='order_id', right_on='orderId', how='left')
    logging.info("Merged orders and invoicing data.")
    
    # Calcular las comisiones y ordenar por total en orden descendente
    commission_df = calculate_commissions(merged_df)
    commission_df = commission_df.sort_values(by='total_commission', ascending=False).reset_index(drop=True)
    commission_df['total_commission'] = commission_df['total_commission'].round(2)
    
    # Mostrar el DataFrame resultante
    logging.info("Sales Owner Commissions:")
    print(commission_df)

# Ejecución principal
if __name__ == "__main__":
    main()

2024-11-05 21:06:42,145 - INFO - Loaded invoicing data with 12 rows.
2024-11-05 21:06:42,148 - INFO - Processed invoicing data with net values in euros.
2024-11-05 21:06:42,151 - INFO - Loaded orders data with 62 rows.
2024-11-05 21:06:42,152 - INFO - Merged orders and invoicing data.
2024-11-05 21:06:42,155 - INFO - Sales Owner Commissions:


          sales_owner  total_commission
0       Leonard Cohen            629.28
1     David Henderson            465.34
2      Luke Skywalker            366.54
3        Yuri Gagarin            273.49
4        David Goliat            250.32
5      Ammy Winehouse            201.72
6   Marianov Merschik            188.61
7         Chris Pratt            112.90
8     Vladimir Chukov             72.83
9         Marie Curie             67.41
10        Leon Leonov             57.31
11       Markus Söder             21.92


## Test 5 DataFrame of Companies with Sales Owners


In [12]:
# Cargar los datos
orders_filepath = '../data/orders.csv'

# Cargar el archivo CSV de órdenes
orders_df = pd.read_csv(orders_filepath, delimiter=';')

# Crear un DataFrame raw con company_id, company_name y salesowners
raw_df = orders_df[['company_id', 'company_name', 'salesowners']].copy()

# Normalización de nombres de las compañías
def normalize_company_name(name):
    if pd.isna(name):
        return ""
    # Convertir a minúsculas
    name = name.lower()
    # Eliminar sufijos comunes y caracteres no deseados
    name = re.sub(r'\b(co|inc|ltd|gmbh|c\.o|limited|corporation|s\.a\.r\.l)\b', '', name)
    name = re.sub(r'[^\w\s]', '', name)  # Eliminar puntuaciones
    name = re.sub(r'\s+', ' ', name).strip()  # Eliminar espacios adicionales
    sorted_words = ' '.join(sorted(name.split()))
    return sorted_words

# Aplicar normalización a company_name
raw_df['normalized_company_name'] = raw_df['company_name'].apply(normalize_company_name)



In [13]:
# show raw_df['normalized_company_name'] value counts and order bn company name
raw_df['normalized_company_name'].value_counts().sort_index().to_clipboard()

# try it using lammatizer

In [14]:
import re
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import pdist, squareform
import pandas as pd

# Descargar datos de WordNet si no se han descargado
nltk.download('wordnet')
nltk.download('omw-1.4')

# Inicializar lematizador de WordNet
lemmatizer = WordNetLemmatizer()

# Función para normalizar el nombre de la compañía
def normalize_company_name(name):
    """Lematiza, limpia y normaliza el nombre de una compañía."""
    if pd.isna(name):
        return ""
    
    # Convertir a minúsculas
    name = name.lower()
    
    # Eliminar sufijos comunes y caracteres no deseados
    name = re.sub(r'\b(co|inc|ltd|gmbh|c\.o|limited|corporation|s\.a\.r\.l)\b', '', name)
    name = re.sub(r'[^\w\s]', '', name)  # Eliminar puntuación
    name = re.sub(r'\s+', ' ', name).strip()  # Eliminar espacios adicionales
    
    # Dividir en palabras, ordenar alfabéticamente y aplicar lematización
    words = sorted(name.split())
    lemmatized_words = [lemmatizer.lemmatize(word, pos='n') for word in words]  # Lematizar como sustantivos
    normalized_name = ' '.join(lemmatized_words)
    
    return normalized_name

# Cargar y procesar datos
def load_and_process_data(filepath):
    """Carga y procesa los datos del archivo CSV de órdenes."""
    orders_df = pd.read_csv(filepath, delimiter=';')
    raw_df = orders_df[['company_id', 'company_name', 'salesowners']].copy()
    raw_df['normalized_company_name'] = raw_df['company_name'].apply(normalize_company_name)
    return raw_df

# Crear matriz de similitud Jaccard
def compute_jaccard_similarity(df, threshold=0.7):
    """Computa la similitud de Jaccard y asigna grupos de similitud."""
    count_vectorizer = CountVectorizer(analyzer='word', binary=True)
    count_matrix = count_vectorizer.fit_transform(df['normalized_company_name'])
    
    # Calcular matriz de distancias Jaccard y convertir a similitud
    jaccard_distances = pdist(count_matrix.toarray(), metric='jaccard')
    jaccard_sim_matrix = 1 - squareform(jaccard_distances)
    
    # Asignar grupos de similitud
    df['similarity_group'] = None
    group_counter = 0

    for i in range(len(jaccard_sim_matrix)):
        if df.loc[i, 'similarity_group'] is None:
            group_counter += 1
            df.loc[i, 'similarity_group'] = group_counter
        for j in range(i + 1, len(jaccard_sim_matrix)):
            if jaccard_sim_matrix[i, j] >= threshold:
                df.loc[j, 'similarity_group'] = df.loc[i, 'similarity_group']
    
    return df

# Ejecutar el procesamiento
orders_filepath = '../data/orders.csv'
raw_df = load_and_process_data(orders_filepath)
processed_df = compute_jaccard_similarity(raw_df)

# Mostrar resultados
processed_df[['normalized_company_name', 'similarity_group']].sort_values(by='normalized_company_name')

[nltk_data] Downloading package wordnet to /Users/pedro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/pedro/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,normalized_company_name,similarity_group
11,berry fresh,9
39,choice healthy,34
30,eats healthy,26
12,express meat,10
22,express seafood,19
...,...,...
55,seafood supplier,3
14,unlimited veggie,12
53,veggie,2
1,veggie,2


In [15]:
processed_df

,company_id,company_name,salesowners,normalized_company_name,similarity_group
0,1e2b47e6-499e-41c6-91d3-09d12dddfbbd,Fresh Fruits Co,"Leonard Cohen, Luke Skywalker, Ammy Winehouse",fresh fruit,1
1,0f05a8f1-2bdf-4be7-8c82-4c9b58f04898,Veggies Inc,"Luke Skywalker, David Goliat, Leon Leonov",veggie,2
2,1e2b47e6-499e-41c6-91d3-09d12dddfbbd,Fresh Fruits c.o,Luke Skywalker,fresh fruit,1
3,1c4b0b50-1d5d-463a-b56e-1a6fd3aeb7d6,Seafood Supplier,"David Goliat, Leonard Cohen",seafood supplier,3
4,34538e39-cd2e-4641-8d24-3c94146e6f16,Meat Packers Ltd,"Chris Pratt, David Henderson, Marianov Merschi...",meat packer,4
...,...,...,...,...,...
57,5f0bdbdf-1d84-4c23-957c-8bb8c0ddc89d,Tropical Fruits Ltd,"Yuri Gagarin, David Goliat",fruit tropical,7
58,acdb6f30-764f-404e-8b8e-7e7e3e6fa1a9,Organic Farms,"Leon Leonov, Chris Pratt",farm organic,6
59,2e90f2b1-d237-47a6-96e8-6d01c0d78c3e,Seafood Supplier GmbH,"Ammy Winehouse, Leonard Cohen",seafood supplier,3
60,fa14c3ed-3c48-49f4-bd69-4d7f5b5f4b1b,Green Veg Co,"Luke Skywalker, Chris Pratt",green veg,5


In [16]:
# Agrupamos por similarity_group para consolidar empresas duplicadas
grouped_df = raw_df.groupby(['similarity_group']).agg({
    'company_id': 'first',  # Escogemos un ID representativo del grupo
    'normalized_company_name': 'first',  # Usamos el nombre normalizado como nombre representativo
    'salesowners': lambda x: sorted(set(', '.join(x).split(', ')))  # Combina, deduplica y ordena vendedores
}).reset_index()

# Convertimos la lista de salesowners en una cadena separada por comas y ordenada alfabéticamente
grouped_df['list_salesowners'] = grouped_df['salesowners'].apply(lambda x: ', '.join(x))

# Renombramos 'normalized_company_name' a 'company_name' para el DataFrame final
df_3 = grouped_df[['company_id', 'normalized_company_name', 'list_salesowners']].rename(columns={'normalized_company_name': 'company_name'})

# Mostrar el resultado final
df_3.sort_values(by='company_name')

,company_id,company_name,list_salesowners
8,4a98d9ec-65f6-438f-9a0c-0d9e1a6f7c65,berry fresh,"Ammy Winehouse, Leonard Cohen"
33,d66c0c95-1f86-4d55-9245-bfa98c0f8dcb,choice healthy,"Ammy Winehouse, Chris Pratt, Leon Leonov"
25,0d09ae2b-d9a5-4d67-bb97-963be9379b4e,eats healthy,"Ammy Winehouse, Yuri Gagarin"
9,f3b70f8d-bb74-4d96-b1f2-88e5c0c68b14,express meat,"Chris Pratt, Marie Curie, Yuri Gagarin"
18,a32972e9-d9ad-4180-9b79-4d29a7d22e10,express seafood,"Leonard Cohen, Yuri Gagarin"
13,5c17d142-4b21-4293-8a34-8dcd2bc24f82,farm fresh,"Ammy Winehouse, Chris Pratt, David Goliat, Leo..."
26,4a7561b1-1de1-420a-93ed-2c12a5bbd1ab,farm global,"David Goliat, Leonard Cohen"
16,1f06b236-f6b5-4f48-8c1c-7b55b5c7d6f7,farm green,"Ammy Winehouse, Leon Leonov"
5,acdb6f30-764f-404e-8b8e-7e7e3e6fa1a9,farm organic,"Chris Pratt, David Henderson, Leon Leonov, Leo..."
32,6cbfd9da-14d3-4a56-b928-3a1e2c71ef83,farm tropical,"David Goliat, Leonard Cohen"


In [17]:
#print dataframe
df_3



,company_id,company_name,list_salesowners
0,1e2b47e6-499e-41c6-91d3-09d12dddfbbd,fresh fruit,"Ammy Winehouse, David Henderson, Leon Leonov, ..."
1,0f05a8f1-2bdf-4be7-8c82-4c9b58f04898,veggie,"David Goliat, Leon Leonov, Leonard Cohen, Luke..."
2,1c4b0b50-1d5d-463a-b56e-1a6fd3aeb7d6,seafood supplier,"Ammy Winehouse, David Goliat, Leonard Cohen, M..."
3,34538e39-cd2e-4641-8d24-3c94146e6f16,meat packer,"Chris Pratt, David Henderson, Leon Leonov, Mar..."
4,fa14c3ed-3c48-49f4-bd69-4d7f5b5f4b1b,green veg,"Ammy Winehouse, Chris Pratt, David Henderson, ..."
5,acdb6f30-764f-404e-8b8e-7e7e3e6fa1a9,farm organic,"Chris Pratt, David Henderson, Leon Leonov, Leo..."
6,5f0bdbdf-1d84-4c23-957c-8bb8c0ddc89d,fruit tropical,"David Goliat, David Henderson, Yuri Gagarin"
7,27c59f76-5d26-4b82-a89b-59f8dfd2e9a7,healthy snack,"Ammy Winehouse, Chris Pratt, Luke Skywalker, M..."
8,4a98d9ec-65f6-438f-9a0c-0d9e1a6f7c65,berry fresh,"Ammy Winehouse, Leonard Cohen"
9,f3b70f8d-bb74-4d96-b1f2-88e5c0c68b14,express meat,"Chris Pratt, Marie Curie, Yuri Gagarin"


## Test 6 

+ What is the distribtion of orders by crate type.
+ Which sales owners need most training to improve selling on plastic crates, based on the last 12 months orders.
+ Understand who are by month the top 5 performers selling plastic crates for a rolling 3 months evaluation window


6.1. What is the distribtion of orders by crate type.

In [218]:
#read in output crate_distribution.csv
crate_distribution = pd.read_csv('../output/crate_distribution.csv')


crate_distribution.crate_type.value_counts()

crate_type
Plastic    19
Metal      14
Wood       13
Name: count, dtype: int64

In [18]:
import pandas as pd

# Load the orders data
orders_filepath = '../data/orders.csv' # Replace with actual path if necessary
orders_df = pd.read_csv(orders_filepath, delimiter=';')

# Convert the date column to datetime
orders_df['date'] = pd.to_datetime(orders_df['date'], format='%d.%m.%y')

# Display basic info and first rows
print(orders_df.info())

orders_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   order_id      62 non-null     object        
 1   date          62 non-null     datetime64[ns]
 2   company_id    62 non-null     object        
 3   company_name  62 non-null     object        
 4   crate_type    62 non-null     object        
 5   contact_data  45 non-null     object        
 6   salesowners   62 non-null     object        
dtypes: datetime64[ns](1), object(6)
memory usage: 3.5+ KB
None


,order_id,date,company_id,company_name,crate_type,contact_data,salesowners
0,f47ac10b-58cc-4372-a567-0e02b2c3d479,2022-01-29,1e2b47e6-499e-41c6-91d3-09d12dddfbbd,Fresh Fruits Co,Plastic,"[{ ""contact_name"":""Curtis"", ""contact_surname"":...","Leonard Cohen, Luke Skywalker, Ammy Winehouse"
1,f47ac10b-58cc-4372-a567-0e02b2c3d480,2022-02-21,0f05a8f1-2bdf-4be7-8c82-4c9b58f04898,Veggies Inc,Wood,"[{ ""contact_name"":""Maria"", ""contact_surname"":""...","Luke Skywalker, David Goliat, Leon Leonov"
2,f47ac10b-58cc-4372-a567-0e02b2c3d481,2022-04-03,1e2b47e6-499e-41c6-91d3-09d12dddfbbd,Fresh Fruits c.o,Metal,"[{ ""contact_name"":""Para"", ""contact_surname"":""C...",Luke Skywalker
3,f47ac10b-58cc-4372-a567-0e02b2c3d482,2021-07-14,1c4b0b50-1d5d-463a-b56e-1a6fd3aeb7d6,Seafood Supplier,Plastic,NaN,"David Goliat, Leonard Cohen"
4,f47ac10b-58cc-4372-a567-0e02b2c3d483,2022-10-23,34538e39-cd2e-4641-8d24-3c94146e6f16,Meat Packers Ltd,Plastic,NaN,"Chris Pratt, David Henderson, Marianov Merschi..."


In [212]:
import plotly.express as px

# Count the number of each crate type
crate_counts = orders_df['crate_type'].value_counts().reset_index()
crate_counts.columns = ['crate_type', 'count']

# Calculate the specific percentage for each crate type
total_orders = crate_counts['count'].sum()
crate_counts['percentage'] = (
    crate_counts['count'] / total_orders * 100).round(2)

# Create the bar chart
fig_crate_distribution = px.bar(
    crate_counts,
    x='crate_type',
    y='count',
    title='Distribution of Orders by Crate Type',
    labels={
        'crate_type': 'Crate Type',
        'count': 'Number of Orders',
        'percentage': 'Percentage'
    },
    color='crate_type',
    color_discrete_sequence=px.colors.qualitative.Pastel
)

# Update traces to show both the number of orders and the percentage in the hover
fig_crate_distribution.update_traces(
    # Hover labels with all the information
    hovertemplate='<b>%{x}</b><br>Orders: %{y}<br>Percentage: %{customdata}%',
    # Pass the percentage as additional data
    customdata=crate_counts['percentage'],
)

# Improve the chart layout
fig_crate_distribution.update_layout(
    title={'x': 0.5, 'xanchor': 'center'},
    xaxis_title='Crate Type',
    yaxis_title='Number of Orders',
    template='plotly_white',
    font=dict(size=16),
)

# Show the chart
fig_crate_distribution.show()

6.2 Which sales owners need most training to improve selling on plastic crates, based on the last 12 months orders.



In [150]:
import pandas as pd
from datetime import timedelta
import json

# Load file paths
orders_filepath = '../data/orders.csv'
invoices_filepath = '../data/invoicing_data.json'

# Load the CSV file of orders
orders_df = pd.read_csv(orders_filepath, delimiter=';')

# Load the JSON file of invoices
with open(invoices_filepath, 'r') as f:
    invoicing_data = json.load(f)["data"]["invoices"]

# Convert JSON data to a DataFrame
invoices_df = pd.json_normalize(invoicing_data)

# Ensure date column is in datetime format in orders
orders_df['date'] = pd.to_datetime(orders_df['date'], format='%d.%m.%y')

# Convert 'grossValue' and 'vat' columns to numeric types
invoices_df['grossValue'] = pd.to_numeric(invoices_df['grossValue'], errors='coerce')
invoices_df['vat'] = pd.to_numeric(invoices_df['vat'], errors='coerce')

# Merge Orders and Invoices Data
merged_df = orders_df.merge(invoices_df[['orderId', 'grossValue', 'vat']], left_on='order_id', right_on='orderId', how='left')

# Fill missing values in `grossValue` and `vat` with zero
merged_df['grossValue'] = merged_df['grossValue'].fillna(0)
merged_df['vat'] = merged_df['vat'].fillna(0)

# Assign "Unknown" to missing values in `salesowners`
merged_df['salesowners'] = merged_df['salesowners'].fillna("Unknown")

# Filter to keep only rows with valid values in orderId
merged_df = merged_df[merged_df['orderId'].notna()]

# Remove duplicates based on 'order_id' and keep only the first occurrence
merged_df_unique = merged_df.drop_duplicates(subset='order_id', keep='first')

# 1. Filter the dataset for orders from the last 12 months
latest_date = merged_df_unique['date'].max()  # Find the most recent order date
one_year_ago = latest_date - timedelta(days=365)
last_12_months_df = merged_df_unique[merged_df_unique['date'] >= one_year_ago]

# 2. Further filter to keep only plastic crate orders
plastic_orders_df = last_12_months_df[last_12_months_df['crate_type'] == 'Plastic'].copy()

# Step 1: Split the salesowners into separate rows
plastic_orders_df['salesowners'] = plastic_orders_df['salesowners'].str.split(', ')
expanded_orders_df = plastic_orders_df.explode('salesowners')

# Step 2: Calculate gross value per salesowner
# Count the number of salesowners for each order_id to calculate individual allocation
expanded_orders_df['salesowner_count'] = expanded_orders_df.groupby('order_id')['salesowners'].transform('count')
expanded_orders_df['gross_per_salesowner'] = expanded_orders_df['grossValue'] / expanded_orders_df['salesowner_count']

# Step 3: Round gross value per salesowner
expanded_orders_df['gross_per_salesowner'] = expanded_orders_df['gross_per_salesowner'].round(2)

# Step 4: Aggregate sales performance by summing gross per salesowner
sales_performance = expanded_orders_df.groupby('salesowners')['gross_per_salesowner'].sum().reset_index()

# Step 5: Sort to identify salesowners with the lowest sales
sales_performance = sales_performance.sort_values(by='gross_per_salesowner')

# Display the sales performance data to identify those needing training
print("Sales performance of each salesowner for plastic crates (last 12 months):")
print(sales_performance)


Sales performance of each salesowner for plastic crates (last 12 months):
         salesowners  gross_per_salesowner
4        Leon Leonov              36366.75
7  Marianov Merschik              36366.75
5      Leonard Cohen             108074.00
6     Luke Skywalker             108074.00
8        Marie Curie             113771.67
2       David Goliat             115103.33
9       Yuri Gagarin             115103.33
1        Chris Pratt             150138.42
3    David Henderson             151470.08
0     Ammy Winehouse             221845.67


In [151]:
import pandas as pd
from datetime import timedelta
import json
import os

# Function to load data with validation
def load_data(file_path, file_type='csv', delimiter=';'):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file '{file_path}' does not exist.")
    
    if file_type == 'csv':
        return pd.read_csv(file_path, delimiter=delimiter)
    elif file_type == 'json':
        with open(file_path, 'r') as f:
            return json.load(f)["data"]["invoices"]
    else:
        raise ValueError("Unsupported file type. Choose 'csv' or 'json'.")

# Function to validate columns and data types
def validate_and_prepare_data(orders_df, invoices_df):
    # Check and parse date column
    if 'date' not in orders_df.columns:
        raise KeyError("Column 'date' not found in orders data.")
    orders_df['date'] = pd.to_datetime(orders_df['date'], format='%d.%m.%y', errors='coerce')
    
    # Convert 'grossValue' and 'vat' columns to numeric types
    for col in ['grossValue', 'vat']:
        if col in invoices_df.columns:
            invoices_df[col] = pd.to_numeric(invoices_df[col], errors='coerce')
        else:
            raise KeyError(f"Column '{col}' not found in invoices data.")
    
    return orders_df, invoices_df

# Function to merge and clean data
def merge_and_clean_data(orders_df, invoices_df):
    merged_df = orders_df.merge(
        invoices_df[['orderId', 'grossValue', 'vat']], 
        left_on='order_id', right_on='orderId', how='left'
    )
    
    # Handle missing values
    merged_df['grossValue'] = merged_df['grossValue'].fillna(0)
    merged_df['vat'] = merged_df['vat'].fillna(0)
    merged_df['salesowners'] = merged_df['salesowners'].fillna("Unknown")
    
    # Remove rows with missing orderId and duplicates
    merged_df = merged_df[merged_df['orderId'].notna()]
    merged_df_unique = merged_df.drop_duplicates(subset='order_id', keep='first')
    
    return merged_df_unique

# Function to filter plastic orders in the last 12 months
def filter_plastic_orders(merged_df):
    latest_date = merged_df['date'].max()
    one_year_ago = latest_date - timedelta(days=365)
    last_12_months_df = merged_df[merged_df['date'] >= one_year_ago]
    plastic_orders_df = last_12_months_df[last_12_months_df['crate_type'] == 'Plastic'].copy()
    
    return plastic_orders_df

# Function to calculate gross value per salesowner
def calculate_sales_performance(plastic_orders_df):
    # Split salesowners and expand rows
    plastic_orders_df['salesowners'] = plastic_orders_df['salesowners'].str.split(', ')
    expanded_orders_df = plastic_orders_df.explode('salesowners')
    
    # Calculate gross value per salesowner
    expanded_orders_df['salesowner_count'] = expanded_orders_df.groupby('order_id')['salesowners'].transform('count')
    expanded_orders_df['gross_per_salesowner'] = (expanded_orders_df['grossValue'] / expanded_orders_df['salesowner_count']).round(2)
    
    # Aggregate sales performance
    sales_performance = expanded_orders_df.groupby('salesowners')['gross_per_salesowner'].sum().reset_index()
    sales_performance = sales_performance.sort_values(by='gross_per_salesowner')
    
    return sales_performance

# Main script execution
if __name__ == "__main__":
    # Load data
    orders_filepath = '../data/orders.csv'
    invoices_filepath = '../data/invoicing_data.json'
    
    orders_df = load_data(orders_filepath, file_type='csv')
    invoicing_data = load_data(invoices_filepath, file_type='json')
    invoices_df = pd.json_normalize(invoicing_data)
    
    # Validate and prepare data
    orders_df, invoices_df = validate_and_prepare_data(orders_df, invoices_df)
    
    # Merge and clean data
    merged_df = merge_and_clean_data(orders_df, invoices_df)
    
    # Filter for plastic orders in the last 12 months
    plastic_orders_df = filter_plastic_orders(merged_df)
    
    # Calculate and display sales performance
    sales_performance = calculate_sales_performance(plastic_orders_df)
    print("Sales performance of each salesowner for plastic crates (last 12 months):")
    print(sales_performance)

Sales performance of each salesowner for plastic crates (last 12 months):
         salesowners  gross_per_salesowner
4        Leon Leonov              36366.75
7  Marianov Merschik              36366.75
5      Leonard Cohen             108074.00
6     Luke Skywalker             108074.00
8        Marie Curie             113771.67
2       David Goliat             115103.33
9       Yuri Gagarin             115103.33
1        Chris Pratt             150138.42
3    David Henderson             151470.08
0     Ammy Winehouse             221845.67


In [174]:
import plotly.express as px

# Redondear y convertir a 'k' el valor de gross_per_salesowner
sales_performance['gross_per_salesowner_k'] = (sales_performance['gross_per_salesowner'] / 1000).round(2)

# Crear un gráfico de barras horizontales en Plotly usando el DataFrame sales_performance
fig = px.bar(
    sales_performance.sort_values(by='gross_per_salesowner'),
    x='gross_per_salesowner_k',
    y='salesowners',
    orientation='h',
    title="Sales Performance for Plastic Crates (Last 12 Months)",
    labels={'salesowners': 'Sales Owners', 'gross_per_salesowner_k': 'Gross Value per Salesowner (k€)'},
    text='gross_per_salesowner_k',
    color='salesowners',
    color_discrete_sequence=px.colors.qualitative.Pastel  # Usar la misma paleta pastel
)

# Estilizar el gráfico para mejorar la legibilidad y el diseño
fig.update_traces(texttemplate='%{text}k', textposition='outside', textfont_size=10)  # Ajustar tamaño de la letra a 10
fig.update_layout(
    title_font_size=18,
    xaxis_title="Gross Value per Salesowner (k€)",
    yaxis_title="",
    xaxis=dict(
        tickformat=",", 
        tickprefix="€", 
        ticksuffix="k",
        range=[0, sales_performance['gross_per_salesowner_k'].max() * 1.2]  # Extender el rango de x un 20% adicional
    ),
    template='plotly_white',
    width=1200,  # Aumentar el ancho del lienzo
    height=700,  # Opcionalmente aumentar el alto también
    showlegend=False
)

fig.show()

6.3 Understand who are by month the top 5 performers selling plastic crates for a rolling 3 months evaluation window


In [203]:
import pandas as pd
import json

# Load file paths
orders_filepath = '../data/orders.csv'
invoices_filepath = '../data/invoicing_data.json'

# Load the CSV file of orders
orders_df = pd.read_csv(orders_filepath, delimiter=';')

# Load the JSON file of invoices
with open(invoices_filepath, 'r') as f:
    invoicing_data = json.load(f)["data"]["invoices"]

# Convert JSON data to a DataFrame
invoices_df = pd.json_normalize(invoicing_data)

# Ensure date column is in datetime format in orders
orders_df['date'] = pd.to_datetime(orders_df['date'], format='%d.%m.%y')

# Convert 'grossValue' and 'vat' columns to numeric types
invoices_df['grossValue'] = pd.to_numeric(invoices_df['grossValue'], errors='coerce')
invoices_df['vat'] = pd.to_numeric(invoices_df['vat'], errors='coerce')

# Merge Orders and Invoices Data
merged_df = orders_df.merge(invoices_df[['orderId', 'grossValue', 'vat']], left_on='order_id', right_on='orderId', how='left')

# Fill missing values in `grossValue` and `vat` with zero
merged_df['grossValue'] = merged_df['grossValue'].fillna(0)
merged_df['vat'] = merged_df['vat'].fillna(0)

# Assign "Unknown" to missing values in `salesowners`
merged_df['salesowners'] = merged_df['salesowners'].fillna("Unknown")

# Remove duplicates based on 'order_id' and keep only the first occurrence
merged_df_unique = merged_df.drop_duplicates(subset='order_id', keep='first')

# Filter for plastic crate orders only
plastic_orders_df = merged_df_unique[merged_df_unique['crate_type'] == 'Plastic'].copy()

# Remove rows where orderId is NaN
plastic_orders_df = plastic_orders_df.dropna(subset=['orderId']).copy()

# Separate salesowners into individual rows
plastic_orders_df['salesowners'] = plastic_orders_df['salesowners'].str.split(', ')
expanded_orders_df = plastic_orders_df.explode('salesowners').copy()

# Calculate gross sales per salesowner for each order
expanded_orders_df['salesowner_count'] = expanded_orders_df.groupby('order_id')['salesowners'].transform('count')
expanded_orders_df['gross_per_salesowner'] = expanded_orders_df['grossValue'] / expanded_orders_df['salesowner_count']

# Create a 'year_month' column in YYYY-MM format
expanded_orders_df['year_month'] = expanded_orders_df['date'].dt.to_period('M').astype(str)

# Aggregate monthly sales performance per salesowner
monthly_sales_performance = (
    expanded_orders_df.groupby(['salesowners', 'year_month'])['gross_per_salesowner']
    .sum()
    .reset_index()
)

# Convert 'year_month' to a datetime index for rolling calculation
monthly_sales_performance['year_month'] = pd.to_datetime(monthly_sales_performance['year_month'], format='%Y-%m')
monthly_sales_performance = monthly_sales_performance.sort_values(by=['salesowners', 'year_month'])

# Calculate 3-month rolling sum for gross sales
monthly_sales_performance['gross_rolling_3m'] = (
    monthly_sales_performance.groupby('salesowners')['gross_per_salesowner']
    .rolling(window=3, min_periods=1)
    .sum()
    .reset_index(drop=True)
)

# Convert 'year_month' back to a period format for display
monthly_sales_performance['year_month'] = monthly_sales_performance['year_month'].dt.to_period('M').astype(str)

# Sort by 'year_month' and 'gross_rolling_3m' in descending order to rank within each month
monthly_sales_performance = (
    monthly_sales_performance.sort_values(by=['year_month', 'gross_rolling_3m'], ascending=[True, False])
)

# Rank the salesowners within each month
monthly_sales_performance['rank'] = monthly_sales_performance.groupby('year_month')['gross_rolling_3m'].rank(method='first', ascending=False)

# Filter to keep only the top 5 for each month
top_5_performers_3m = monthly_sales_performance[monthly_sales_performance['rank'] <= 5]

# Display the final table with top 5 performers per month
print("Top 5 performers selling plastic crates for each 3-month rolling window:")
print(top_5_performers_3m[['year_month', 'salesowners', 'gross_rolling_3m', 'rank']])

Top 5 performers selling plastic crates for each 3-month rolling window:
   year_month        salesowners  gross_rolling_3m  rank
4     2021-07       David Goliat     122706.000000   1.0
9     2021-07      Leonard Cohen     122706.000000   2.0
10    2022-01      Leonard Cohen     230780.000000   1.0
0     2022-01     Ammy Winehouse     108074.000000   2.0
11    2022-01     Luke Skywalker     108074.000000   3.0
5     2022-09       David Goliat     237809.333333   1.0
6     2022-09    David Henderson     115103.333333   2.0
14    2022-09       Yuri Gagarin     115103.333333   3.0
7     2022-10    David Henderson     151470.083333   1.0
2     2022-10        Chris Pratt      36366.750000   2.0
8     2022-10        Leon Leonov      36366.750000   3.0
12    2022-10  Marianov Merschik      36366.750000   4.0
1     2022-11     Ammy Winehouse     221845.666667   1.0
3     2022-11        Chris Pratt     150138.416667   2.0
13    2022-11        Marie Curie     113771.666667   3.0


In [220]:
import pandas as pd
import json
from datetime import timedelta

# Function to load orders data
def load_orders_data(filepath):
    try:
        orders_df = pd.read_csv(filepath, delimiter=';')
        orders_df['date'] = pd.to_datetime(orders_df['date'], format='%d.%m.%y', errors='coerce')
        if orders_df['date'].isnull().any():
            raise ValueError("Some dates could not be parsed.")
        return orders_df
    except Exception as e:
        raise RuntimeError(f"Error loading orders data: {e}")

# Function to load invoicing data
def load_invoices_data(filepath):
    try:
        with open(filepath, 'r') as f:
            invoicing_data = json.load(f).get("data", {}).get("invoices", [])
            if not invoicing_data:
                raise ValueError("No invoices data found.")
        invoices_df = pd.json_normalize(invoicing_data)
        invoices_df['grossValue'] = pd.to_numeric(invoices_df['grossValue'], errors='coerce').fillna(0)
        invoices_df['vat'] = pd.to_numeric(invoices_df['vat'], errors='coerce').fillna(0)
        return invoices_df
    except Exception as e:
        raise RuntimeError(f"Error loading invoicing data: {e}")

# Function to merge data and prepare the dataset
def prepare_data(orders_df, invoices_df):
    try:
        merged_df = orders_df.merge(invoices_df[['orderId', 'grossValue', 'vat']], left_on='order_id', right_on='orderId', how='left')
        merged_df['grossValue'] = merged_df['grossValue'].fillna(0)
        merged_df['vat'] = merged_df['vat'].fillna(0)
        merged_df['salesowners'] = merged_df['salesowners'].fillna("Unknown")
        merged_df_unique = merged_df.drop_duplicates(subset='order_id', keep='first')
        plastic_orders_df = merged_df_unique[merged_df_unique['crate_type'] == 'Plastic'].dropna(subset=['orderId']).copy()
        return plastic_orders_df
    except KeyError as e:
        raise KeyError(f"Missing expected column: {e}")
    except Exception as e:
        raise RuntimeError(f"Error preparing data: {e}")

# Function to expand and calculate gross per salesowner
def expand_salesowners(plastic_orders_df):
    try:
        plastic_orders_df['salesowners'] = plastic_orders_df['salesowners'].str.split(', ')
        expanded_orders_df = plastic_orders_df.explode('salesowners').copy()
        expanded_orders_df['salesowner_count'] = expanded_orders_df.groupby('order_id')['salesowners'].transform('count')
        expanded_orders_df['gross_per_salesowner'] = (expanded_orders_df['grossValue'] / expanded_orders_df['salesowner_count']).round(2)
        expanded_orders_df['year_month'] = expanded_orders_df['date'].dt.to_period('M').astype(str)
        return expanded_orders_df
    except Exception as e:
        raise RuntimeError(f"Error expanding salesowners data: {e}")

# Function to calculate top performers for a rolling 3-month period
def calculate_top_performers(expanded_orders_df):
    try:
        monthly_sales_performance = (
            expanded_orders_df.groupby(['salesowners', 'year_month'])['gross_per_salesowner']
            .sum()
            .reset_index()
        )
        monthly_sales_performance['year_month'] = pd.to_datetime(monthly_sales_performance['year_month'], format='%Y-%m')
        monthly_sales_performance = monthly_sales_performance.sort_values(by=['salesowners', 'year_month'])
        
        monthly_sales_performance['gross_rolling_3m'] = (
            monthly_sales_performance.groupby('salesowners')['gross_per_salesowner']
            .rolling(window=3, min_periods=1)
            .sum()
            .reset_index(drop=True)
        )
        
        monthly_sales_performance['year_month'] = monthly_sales_performance['year_month'].dt.to_period('M').astype(str)
        top_5_performers_3m = (
            monthly_sales_performance.sort_values(by=['year_month', 'gross_rolling_3m'], ascending=[True, False])
            .groupby('year_month')
            .head(5)
        )
        top_5_performers_3m['rank'] = top_5_performers_3m.groupby('year_month')['gross_rolling_3m'].rank(ascending=False, method='first')
        # reset index
        top_5_performers_3m = top_5_performers_3m.reset_index(drop=True)
        return top_5_performers_3m
    except Exception as e:
        raise RuntimeError(f"Error calculating top performers: {e}")

# Main function to orchestrate the workflow
def main(orders_filepath, invoices_filepath):
    try:
        orders_df = load_orders_data(orders_filepath)
        invoices_df = load_invoices_data(invoices_filepath)
        plastic_orders_df = prepare_data(orders_df, invoices_df)
        expanded_orders_df = expand_salesowners(plastic_orders_df)
        top_5_performers = calculate_top_performers(expanded_orders_df)
    
        print("Top 5 performers selling plastic crates for each 3-month rolling window:")
        print(top_5_performers)
        return top_5_performers
    except Exception as e:
        print(f"An error occurred: {e}")

# Run the main function
orders_filepath = '../data/orders.csv'  # Update paths as needed
invoices_filepath = '../data/invoicing_data.json'
top_5_performers = main(orders_filepath, invoices_filepath)

Top 5 performers selling plastic crates for each 3-month rolling window:
          salesowners year_month  gross_per_salesowner  gross_rolling_3m  rank
0        David Goliat    2021-07             122706.00         122706.00   1.0
1       Leonard Cohen    2021-07             122706.00         122706.00   2.0
2       Leonard Cohen    2022-01             108074.00         230780.00   1.0
3      Ammy Winehouse    2022-01             108074.00         108074.00   2.0
4      Luke Skywalker    2022-01             108074.00         108074.00   3.0
5        David Goliat    2022-09             115103.33         237809.33   1.0
6     David Henderson    2022-09             115103.33         115103.33   2.0
7        Yuri Gagarin    2022-09             115103.33         115103.33   3.0
8     David Henderson    2022-10              36366.75         151470.08   1.0
9         Chris Pratt    2022-10              36366.75          36366.75   2.0
10        Leon Leonov    2022-10              36366.75    

In [211]:
import plotly.graph_objects as go

# Format the "Gross Rolling 3M" column with a thousands separator
top_5_performers["gross_rolling_3m_formatted"] = top_5_performers["gross_rolling_3m"].map("{:,.2f}".format)

# Create a Plotly Table to display the top 5 performers with thousands separators
def display_top_performers_table(top_5_performers):
    fig = go.Figure(
        data=[
            go.Table(
                header=dict(
                    values=["Year-Month", "Salesowner", "Gross Rolling 3M (€)", "Rank"],
                    fill_color="lightblue",
                    align="center",
                    font=dict(size=12, color="black"),
                ),
                cells=dict(
                    values=[
                        top_5_performers["year_month"],
                        top_5_performers["salesowners"],
                        top_5_performers["gross_rolling_3m_formatted"],
                        top_5_performers["rank"].astype(int),
                    ],
                    fill_color="white",
                    align="center",
                    font=dict(size=11),
                ),
            )
        ]
    )
    fig.update_layout(
        title="Top 5 Performers Selling Plastic Crates for Each 3-Month Rolling Window",
        title_x=0.5,
        margin=dict(l=20, r=20, t=60, b=20),
        width=800,
        height=500,
    )
    fig.show()

# Display the table
display_top_performers_table(top_5_performers)